In [10]:
# pip install ml_metadata

In [ ]:
pip install tensorflow-data-validation

In [3]:
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))

import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))

import urllib
import zipfile

TF version: 2.11.0
TFDV version: 1.11.0


In [4]:
url = 'https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip'
zip, headers = urllib.request.urlretrieve(url)
zipfile.ZipFile(zip).extractall()
zipfile.ZipFile(zip).close()

In [5]:
!ls -R data

data:
eval  serving  train

data/eval:
data.csv

data/serving:
data.csv

data/train:
data.csv
